Este notebook é uma adaptação para uso no ambiente Google Colab do notebook **notebook_02.ipynb** fornecido como material complementar do livro Inteligência Artificial: Uma Abordagem de Aprendizado de Máquina|FACELI, Katti; LORENA, Ana C.; GAMA, João; AL, et. Tendo sido desenvolvido originalmente por: Renato Moraes Silva.

Antes de iniciar este notebook, salve o arquivo do conjunto de dados iris (iris1.csv) em um diretório local na sua máquina

In [ ]:
# O código abaixo monta um drive no Google colab para que você possa fazer o upload do arquivo de dados iris da sua máquina.

# após executar está celula, escolha o arquivo iris1.csv que você salvou na sua máquina
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## Introdução

Neste notebook, além dos algoritmos utilizados no notebook da semana passada (K-NN e Naive Bayes), o algoritmo SVM (Support Vector Machine) e o algoritmo de indução de árvore de decisão também serão treinados e testados para classificação do conjunto de dados iris. Os resultados de desempenho em termos de tempo de execução, e taxa de acertos (acurácia), serão analisados de forma similiar ao que foi feito no notebook da semana anterior, agora juntamente com os resultados para o k-NN e Naive Bayes. Também geraremos a  visualiação da árvore de decisão induzida . Ao final, realizaremos outras medidas de desempenho, além da geração da matriz de confusão relativa a cada um dos resultados de predição obtidos pelos modelos.

Os conceitos apresentados neste notebook fazem referência ao conteúdo abordado nos Capítulos **6 (Métodos Simbólicos)** e **8 (Métodos de Maximização de Márgens)** e **10 (Avaliação de Modelos Preditivos)**

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np # importa a biblioteca usada para trabalhar com vetores e matrizes
import pandas as pd # importa a biblioteca usada para trabalhar com dataframes e análise de dados
import sklearn as skl # importa o sckit-learn
import time # sera usada para calcular o tempo de execucao dos metodos de classificacao

from sklearn import model_selection 
from sklearn import naive_bayes # necessario para usar o metodo naive Bayes
from sklearn import tree # necessario para usar arvores de decisao
from sklearn import svm # necessario para usar o metodo SVM
from sklearn import neighbors # necessario para usar o metodo KNN
from sklearn import metrics # necessario para obter o desempenho da classificacao

# bibliotecas para geração de gráficos
import graphviz # usado para realizar a representação gráfica de umar árvore de decisão
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

print('Bibliotecas carregadas com sucesso')

Vamos carregar os dados do arquivo 

In [2]:
# importa o arquivo e guarda em um dataframe do Pandas
df_dataset = pd.read_csv( 'iris1.csv', sep=',', index_col=None) 

Por questões didáticas, vamos realizar os experimentos considerando apenas os atributos largura (*SepalWidthCm*) e comprimento da sépala (*SepalLengthCm*).

In [3]:
# remove as colunas que não serão usadas 
df_dataset = df_dataset.drop(columns=['Id','PetalLengthCm','PetalWidthCm'])

Vamos plotar os dados.

In [ ]:
# scatter plot
sns.lmplot(x='SepalLengthCm', y='SepalWidthCm', data=df_dataset, 
           fit_reg=False,   # Sem linha de regressão
           hue='Species')   # Cores diferentes por classe

# cria um título para o gráfico
plt.title('SepalLengthCm vs SepalWidthCm')

# mostra o gráfico
plt.show()

Armazena os dados dentro de uma matriz e as classes dentro de um vetor.

In [5]:
# obtém os valores das n-1 primeiras colunas e guarda em uma matrix X
X = df_dataset[['SepalLengthCm','SepalWidthCm']].values 

# obtém os valores da ultima coluna e guarda em um vetor Y
Y = df_dataset[['Species']].values 
Y = np.ravel( Y ) # converte em um vetor

Vamos dividir a base de dados em duas partições (treino e teste), mantendo a distribuição original de dados de cada classe em cada partição. Os primeiros $70\%$ dos dados de cada classe irão compor o conjunto de treinamento, enquanto o restante  comporá os dados de teste. Para isso, usaremos a função `sklearn.model_selection.StratifiedShuffleSpli`. 

A função `sklearn.model_selection.StratifiedShuffleSplit` pode ser usada para gerar $n$ particionamentos estratificados. A quantidade de particionamentos é definida pelo parâmetro `n_splits`.  Vamos usar apenas um particionamento de treino e de teste. Por isso, iremos setar o parâmetro `n_splits` com valor 1. 

É importante que as partições de treinamento e teste sejam geradas de forma aleatória. Para que toda a execução gere o mesmo resultado, vamos usar uma semente para a função de geração de números aleatórios, setando um valor qualquer para o parâmetro `random_state`.

In [ ]:
# gera uma divisão dos dados em treino e teste, com 70% de dados para o treinamento e 30% para teste
cv = skl.model_selection.StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=2020)

# retorna os índices de treino e teste
train_index, test_index = list( cv.split(X, Y) )[0]

# retorna as partições de treino e teste de acordo com os índices
X_train, X_test = X[train_index, :], X[test_index, :];
Y_train, Y_test = Y[train_index], Y[test_index];

print('Qtd. dados de treinamento: %d (%1.2f%%)' %(X_train.shape[0], (X_train.shape[0]/X.shape[0])*100) )
print('Qtd. de dados de teste: %d (%1.2f%%)' %(X_test.shape[0], (X_test.shape[0]/X.shape[0])*100) )

# imprime a porcentagem de dados de treinamento de cada classe
print("\nQtd. de dados de cada classe (treinamento)")
cTrain, counts_cTrain = np.unique(np.sort(Y_train), return_counts=True)
for i in range( len(cTrain) ):
    print('\tClasse %s: %d (%1.2f%%)' %( cTrain[i],counts_cTrain[i],(counts_cTrain[i]/len(Y_train))*100 ) )

# imprime a porcetagem de dados de teste de cada classe
print("\nQtd. de dados de cada classe (teste)")
cTest, counts_cTest = np.unique(np.sort(Y_test), return_counts=True)
for i in range( len(cTrain) ):
    print('\tClasse %s: %d (%1.2f%%)' %( cTest[i],counts_cTest[i],(counts_cTest[i]/len(Y_test))*100 ) )


Vamos normalizar os valores dos atributos para que fiquem com média igual a zero e desvio padrão igual a um. 

In [7]:
# normaliza os valores dos atributos para que fiquem com media igual a zero e desvio padrao igual a um
scaler = skl.preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Agora que separamos os dados em duas partições, podemos treinar os algoritmos de classicação K-NN, Naive Bayes, SVM, e o algoritmo de indução de árvore de decisão  utilizando a partição de treinamento,  e testar utilizando a partição de teste.  

A função **perform_experiment_mod** realiza o treinamento e teste de classificadores, além disso ela também calcula o tempo gasto pelos algoritmos no treinamento e teste, além de fornecer a medida de acurácia. 

In [8]:
def perform_experimen_mod(dataset, target, methodName):
    """
    Função usada para executar os experimentos
    
    Parametros:
    -----------
    dataset: array
        É um array com dimensão "m x n", onde "m" é o número de amostras e "n" é o número de atributos

    target: array
        É uma lista ou um vetor contendo as classes de cada amostra
    
    methodName: string
    
        Um nome usado para identificar o método.

        'G.NB': Naive Bayes Gaussiano
        'KNN': k-nearest neighbors 
        'SVM': Support vector machines
        'DT': Decision trees        
    
    """
    

    resultados=[] # cria uma lista vazia para guardar os resultados obtidos 
  
    auxResults=[]


    x_train, x_test = X_train, X_test
    y_train, y_test =  Y_train,  Y_test
 
    # inicia um timer para analisar o tempo levado para o treinamento e teste
    startTime = time.time() 
            
    #define o classificador que será utilizados
    if methodName == 'G.NB': # Naive Bayes Gaussiano
    # inicia o classificador
      classifier = skl.naive_bayes.GaussianNB()  
    elif methodName == 'KNN': # K-Vizinhos mais próximos
    # inicia o classificador com os parâmetros default do Scikit
      classifier  = skl.neighbors.KNeighborsClassifier()
    elif methodName == 'DT': # Árvores de decisão 
        # inicia o classificador
        classifier = skl.tree.DecisionTreeClassifier(criterion='entropy')
    elif methodName == 'SVM':
        # inicia o classificador com os parâmetros default do Scikit
        classifier = skl.svm.SVC()
        
    # treina o classificador com os dados de treinameto
    classifier.fit(x_train, y_train) 
        
    # classifica os dados de teste
    y_pred = classifier.predict(x_test) 

    # obtem a acuracia
    acuracia = metrics.accuracy_score(y_test, y_pred)         

    # calcula o tempo despendido no treinamento e no teste
    tempo = time.time() - startTime

    # guarda o classificador dentro de resultados
    auxResults=({'classificador': classifier})
    
    # acrescenta o tempo aos resultados
    auxResults.update({'tempo': tempo})


    # acrescenta a acuracia dentro de resultados
    auxResults.update({'acuracia': acuracia})

    # guarda o classificador dentro de resultados
    auxResults.update({'classifier': classifier})

# adiciona os resultados  na lista de resultados
    resultados.append( auxResults ) 
    return resultados
        

Vamos executar o experimento para obtenção dos resultados para os algoritmos. 

In [9]:
metodos = ['G.NB','KNN','DT','SVM'] 

# cria uma lista vazia para guardar as informacoes retornadas em cada experimentos
resultadosMetodos = []

# para cada método da lista de métodos, executa um experimento com os parâmetros informados    
for methodName in metodos:
    resultados = perform_experimen_mod(X, Y, methodName)
    resultadosMetodos.append(resultados)

Observamos que o  KNN  continua sendo o algoritmo mais demorado. Isto se deve, ao que já foi mencionado no notebook da semana anterior, de que o cálculo da vizinhança de cada amostra realizado pelo K-NN ser computacionalmente custoso.

In [ ]:

tempoMetodo = {}

for i, methodName in enumerate(metodos):
    
    auxTempo = [ d['tempo'] for d in resultadosMetodos[i] ] 
    
    tempoMetodo.update({methodName: np.mean(auxTempo)})
 
# cria  um dataframe com os tempos de cada metodo 
df_tempo = pd.DataFrame(tempoMetodo.items(), columns=['Metodo', 'Tempo'])

# definindo o tamanho da figura 
plt.figure(figsize=(10,8))

# cria um gráfico de barras 
sns.barplot(x="Metodo", y="Tempo", data=df_tempo)

# mostra o gráfico
plt.show()

Vamos verificar a acurácia dos modelos gerados

In [ ]:
accMetodo = {}

for i, methodName in enumerate(metodos):
    
    auxAcc = [ d['acuracia'] for d in resultadosMetodos[i] ] 
    
    accMetodo.update({methodName: np.mean(auxAcc)})
 
# cria  um dataframe com os tempos de cada metodo 
df_acc = pd.DataFrame(accMetodo.items(), columns=['Metodo', 'Acuracia'])

# definindo o tamanho da figura 
plt.figure(figsize=(10,8))

# cria um gráfico de barras 
sns.barplot(x="Metodo", y="Acuracia", data=df_acc)

# mostra o gráfico
plt.show()

O a função abaixo é utilizada para plotagem da região de decisão correspondente à classificação dos modelos.

In [12]:
def plota_superficieDecisao(methodName, X, Y, ax, title = ""):
    h = .02  # tamanho do passo da malha (mesh)

    # cria uma malha (mesh)
    x_min, x_max = X[:, 0].min() - 0.3, X[:, 0].max() + 0.3
    y_min, y_max = X[:, 1].min() - 0.3, X[:, 1].max() + 0.3
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
     

    #define o classificador que será utilizados
    if methodName == 'G.NB': # Naive Bayes Gaussiano
    # inicia o classificador
      classifier = skl.naive_bayes.GaussianNB()  
    elif methodName == 'KNN': # K-Vizinhos mais próximos
    # inicia o classificador com os parâmetros default do Scikit
      classifier  = skl.neighbors.KNeighborsClassifier()
    elif methodName == 'DT': # Árvores de decisão 
        # inicia o classificador
        classifier = skl.tree.DecisionTreeClassifier(criterion='entropy')
    elif methodName == 'SVM':
        # inicia o classificador com os parâmetros default do Scikit
        classifier = skl.svm.SVC()
        

    # treina o classificador com os dados de treinameto
    classifier.fit(X_train, Y_train) 
    


    # obtem a predicao
    Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])

    # converte os valores do vetor para indices
    Z2 = np.unique(Z, return_inverse=True)[1]

    # plota a superficie de decisao
    Z2 = Z2.reshape(xx.shape)
    ax.contourf(xx, yy, Z2, cmap=plt.cm.Paired, alpha=.4)

    # converte os valores do vetor para indices
    Y2 = np.unique(Y, return_inverse=True)[1]

    # plota os dados de treinamento
    ax.scatter(X[:, 0], X[:, 1], c=Y2, edgecolor='k', s=50)
    
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_title(title, fontsize='large')
    



Vamos dar uma olhada na superfície de decisão aprendida pelos modelos.

In [ ]:
# define a quantidade de linhas e colunas e o tamanho da figura
nrows = 3; ncols=2; 
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8*ncols, 8*nrows))

# apaga a ultima figura 
fig.delaxes(axs[2,0]) #The indexing is zero-based here
fig.delaxes(axs[2,1])
# converte o array em um vetor
axs = np.concatenate(axs)
    
for i, methodName in enumerate(metodos):
    
    auxClassificador = [ d['classifier'] for d in resultadosMetodos[i] ]
    
    for train_index, test_index in cv.split(X, Y):

        X_train, X_test = X[train_index, :], X[test_index, :];
        Y_train, Y_test = Y[train_index], Y[test_index];
        
        if methodName in ['G.NB', 'SVM', 'DT', 'KNN']:
            
            # normaliza os valores dos atributos para que fiquem com media igual a zero e desvio padrao igual a um
            scaler = skl.preprocessing.StandardScaler().fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
    
        plota_superficieDecisao(methodName, X_test, Y_test, axs[i], title = methodName)
        
        break

**Visualização da Árvore de Decisão**

Abaixo vamos gerar a visualização da árvore de decisão gerada.

Obs1.Em cada nó da árvore de decisão é apresentado (como exemplo seja o primeiro nó):

*o critério condicional para a divisão (e.g. SepalLengthCm < -0.151 )

*o valor da entropia do conjunto (e.g. entropy = 1.585)

*o número de exemplos no conjunto.(e.g. samples = 105)

*o numero de exemplos de cada classe presente no conjunto.(e.g. [35, 35, 35] ->35 Iris-setosa, 35 Iris-versicolor, 35 Iris-Virginica)

*a classe predominante no conjunto

Obs2. Aparecem valores negativos nos valores das dimensões das sépalas ('NormSepalLengthCm','NormSepalWidthCm'), porque o conjunto de dados sofreu regularização antes do processamento.

In [ ]:
classifier = skl.tree.DecisionTreeClassifier(criterion='entropy')
# treina o classificador com os dados de treinameto
classifier.fit(X_train, Y_train)
#representação gráfica da árvore de decisão
dot_data = tree.export_graphviz(classifier, out_file=None, 
feature_names=['NormSepalLengthCm','NormSepalWidthCm'], 
class_names=['Iris-setosa','Iris-versicolor' ,'Iris-virginica' ],
filled=True, rounded=True,  
special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

### Medidas de desempenho

Vamos calcular o desempenho dos algoritmos de classificação (Capítulo 10 - Avaliação de Modelos Preditivos). Algumas medidas de desempenho (*e.g.*, acurácia) retornam um valor global de desempenho, enquanto que outras (*e.g.*, precisão, revocação e F-medida) retornam um valor que pode variar dependendo de qual classe é considerada como positiva (classe alvo do problema). Supondo que em um determinado problema, a classe $c_1$ seja considerada a classe positiva, as seguintes medidas de desempenho podem ser calculadas:

* $\displaystyle \text{acurácia} =\frac{vp_1+vn_1}{vp_1+vn_1+fp_1+fn_1} = \frac{\text{Qtd. de predições corretas}}{\text{Qtd. de amostras}};$

* $\displaystyle \text{revocação} =  \frac{vp_1}{vp_1+fn_1} \text{;} $

* $\displaystyle \text{precisão} = \frac{vp_1}{vp_1+fp_1}; $

* $\displaystyle \text{F-medida} = 2 \times \frac{\text{precisão} \times\text{revocação}}{\text{precisão}+\text{revocação}}.$

Para problemas binários sem uma classe-alvo ou para problemas multiclasse, normalmente são utilizadas medidas de desempenho que consideram a média entre os resultados relativos a cada classe do problema. As duas principais estratégias para obter a média de desempenho entre as classes são a média macro e a média micro. A média macro considera que todas as classes possuem a mesma importância. Por outro lado, na média micro, o resultado final é dominado pelas classes mais frequentes, o que pode gerar um desempenho superestimado quando as classes são muito desbalanceadas. Abaixo, são apresentadas algumas medidas de desempenho calculadas por meio dessas duas estratégias.

* Medidas baseadas na média macro:
 - $ \displaystyle \text{macro revocação} = \frac{1}{{|\mathcal{C}|}} \times \sum_{j=1}^{{|\mathcal{C}|}} \frac{vp_j}{vp_j+fn_j}$;
		
 - $ \displaystyle \text{macro precisão} = \frac{1}{{|\mathcal{C}|}} \times \sum_{j=1}^{{|\mathcal{C}|}} \frac{vp_j}{vp_j+fp_j}$;
		
 - $ \displaystyle \text{macro F-medida} = 2 \times \frac{\text{macro precisão} \times\text{macro revocação}}{\text{macro precisão}+\text{macro revocação}}$.


* Medidas baseadas na média micro:
 - $ \displaystyle \text{micro revocação} = \frac{\sum_{j=1}^{{|\mathcal{C}|}} vp_j}{\sum_{j=1}^{{|\mathcal{C}|}} vp_j+fn_j}$;

 - $ \displaystyle \text{micro precisão} = \frac{\sum_{j=1}^{{|\mathcal{C}|}} vp_j}{\sum_{j=1}^{{|\mathcal{C}|}} vp_j+fp_j}$; 

 - $ \displaystyle \text{micro F-medida} = 2 \times \frac{\text{micro precisão} \times\text{micro revocação}}{\text{micro precisão}+\text{micro revocação}}$.

Na função abaixo, será gerado um relatório com as principais medidas de desempenho. Será calculada a **precisão**, **revocação** e **F-medida** para cada uma das classes do problema. Adicionalmente, será calculada a **acurácia**, **macro** e **micro precisão**, **macro** e **micro revocação** e, por fim, a **macro** e **micro F-medida**.

In [15]:
def relatorioDesempenho(Y_test, Y_pred, classes, imprimeRelatorio=False):
  """
  Funcao usada calcular as medidas de desempenho da classificação.
  
  Parametros
  ----------   
    
  classes: classes do problema
  
  imprimeRelatorio: variavel booleana que indica se o relatorio de desempenho
                    deve ser impresso ou nao. 
     
  Retorno
  -------
  resultados: variavel do tipo dicionario (dictionary). As chaves
              desse dicionario serao os nomes das medidas de desempenho; os valores
              para cada chave serao as medidas de desempenho calculadas na funcao.
              
              Mais especificamente, o dicionario devera conter as seguintes chaves:
              
               - acuracia: valor entre 0 e 1 
               - revocacao: um vetor contendo a revocacao obtida em relacao a cada classe
                            do problema
               - precisao: um vetor contendo a precisao obtida em relacao a cada classe
                            do problema
               - fmedida: um vetor contendo a F-medida obtida em relacao a cada classe
                            do problema
               - revocacao_macroAverage: valor entre 0 e 1
               - precisao_macroAverage: valor entre 0 e 1
               - fmedida_macroAverage: valor entre 0 e 1
               - revocacao_microAverage: valor entre 0 e 1
               - precisao_microAverage: valor entre 0 e 1
               - fmedida_microAverage: valor entre 0 e 1
  """

  # obtem a quantidade de classes
  nClasses = len(classes)

  # obtem a acuracia
  acuracia = metrics.accuracy_score(Y_test, Y_pred)
    
  # inicializa as medidas de desempenho 
  revocacao = np.zeros( len(classes) )
  precisao = np.zeros( len(classes) )
  fmedida = np.zeros( len(classes) )
    
  # calcula a medida de desempenho para cada classe individualmente
  for i in range( len(classes) ):
    
      # transforma o problema multiclasse em binário, apenas para calcular o desempenho individual da classe i
      auxY_test = np.zeros( len(Y_test) ) # inicializa o vetor de classes binárias com 0
      auxY_pred = np.zeros( len(Y_pred) ) # inicializa o vetor de classes binárias com 0
      auxY_test[Y_test==classes[i]] = 1 # onde a classe for igual a classe[i], recebe valor 1
      auxY_pred[Y_pred==classes[i]] = 1 # onde a classe for igual a classe[i], recebe valor 1
        
      revocacao[i] = metrics.recall_score(auxY_test, auxY_pred, pos_label=1) # revocacao
      precisao[i] = metrics.precision_score(auxY_test, auxY_pred, pos_label=1) # precisao
      fmedida[i] = metrics.f1_score(auxY_test, auxY_pred, pos_label=1) # f-medida 
  

  revocacao_microAverage =  metrics.recall_score(Y_test, Y_pred, average='micro')
  precisao_microAverage = metrics.precision_score(Y_test, Y_pred, average='micro')
  fmedida_microAverage = metrics.f1_score(Y_test, Y_pred, average='micro')

  revocacao_macroAverage =  metrics.recall_score(Y_test, Y_pred, average='macro')
  precisao_macroAverage = metrics.precision_score(Y_test, Y_pred, average='macro')
  fmedida_macroAverage = metrics.f1_score(Y_test, Y_pred, average='macro')
  
  
  # imprime os resultados para cada classe
  if imprimeRelatorio:
        
      print('\n\tRevocacao   Precisao   F-medida   Classe')
      for i in range(nClasses):
        print('\t%1.3f       %1.3f      %1.3f      %s' % (revocacao[i], precisao[i], fmedida[i],classes[i] ) )
    
      print('\t------------------------------------------------');
      
      #imprime as médias
      print('\t%1.3f       %1.3f      %1.3f      Média macro' % (revocacao_macroAverage, precisao_macroAverage, fmedida_macroAverage) )
      print('\t%1.3f       %1.3f      %1.3f      Média micro\n' % (revocacao_microAverage, precisao_microAverage, fmedida_microAverage) )
    
      print('\tAcuracia: %1.3f' %acuracia)
      
  # armazena os resultados em uma estrutura tipo dicionario
  resultados = {'revocacao': revocacao, 'acuracia': acuracia, 'precisao':precisao, 'fmedida':fmedida}
  resultados.update({'revocacao_macroAverage':revocacao_macroAverage, 'precisao_macroAverage':precisao_macroAverage, 'fmedida_macroAverage':fmedida_macroAverage})
  resultados.update({'revocacao_microAverage':revocacao_microAverage, 'precisao_microAverage':precisao_microAverage, 'fmedida_microAverage':fmedida_microAverage})

  return resultados 


### Matriz de confusão

A maioria das medidas de desempenho pode ser calculada a partir da matriz de confusão. O código abaixo, irá gerar as predições que serão utilizadas na matriz de confusão para um modelo selecionado

In [16]:
def confusao(methodName):
   #define o classificador que será utilizados
    if methodName == 'G.NB': # Naive Bayes Gaussiano
    # inicia o classificador
      classifier = skl.naive_bayes.GaussianNB()  
    elif methodName == 'KNN': # K-Vizinhos mais próximos
    # inicia o classificador com os parâmetros default do Scikit
      classifier  = skl.neighbors.KNeighborsClassifier()
    elif methodName == 'DT': # Árvores de decisão 
        # inicia o classificador
        classifier = skl.tree.DecisionTreeClassifier(criterion='entropy')
    elif methodName == 'SVM':
        # inicia o classificador com os parâmetros default do Scikit
        classifier = skl.svm.SVC()
        

    # treina o classificador com os dados de treinameto
    classifier.fit(X_train, Y_train)   
    # classifica os dados de teste
    Y_pred = classifier.predict(X_test)
    return Y_pred

**Visualização da Matriz de Confusão e das Medidas de Desempenho**

Vamos agora verificar os dados de desempenho de cada um dos algoritmos juntamente com sua respectiva matriz de confusão.

In [ ]:
# obtem as classes 
classesDataset = np.unique(Y_train)
for i, methodName in enumerate(metodos):

  Y_pred=confusao(methodName)
  # obtem as classes 
  classesDataset = np.unique(Y_train)
  
# obtem a matriz de confusão
  cm = skl.metrics.confusion_matrix(Y_test, Y_pred)

# vamos plotar a matriz de confusao 

# definindo o tamanho da figura 
  plt.figure(figsize=(10,8))
    
# cria um mapa de cores dos valores da matriz de confusao
  sns.heatmap(cm, xticklabels=classesDataset, yticklabels=classesDataset, cmap="YlGnBu", annot=True)
  sns.set(font_scale=1.1) # aumenta a escala das fontes do grafico
        
  plt.xlabel('Classes preditas')
  plt.ylabel('Classes verdadeiras')
  plt.title('Matriz de confusão  '+ methodName)
  plt.show()
  auxResults = relatorioDesempenho(Y_test, Y_pred, classesDataset, imprimeRelatorio=True)
  print('------------------------------------------------------------------');
  print('------------------------------------------------------------------');

---
## Conclusão

Neste notebook, adicionamentos aos algoritmos K-NN e Naive Bayes, os algoritmos SVM de de indução de árvore de decisão, realizando as etapas de treinamento e teste utilizando o conjunto de dados iris. finalmente realizamos diversas medidas de desempenho para a avaliação dos resultados dos modelos. 